In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

end_date = datetime.today()
start_date_daily = datetime(2015, 1, 1)
start_date_hourly = end_date - timedelta(days=90)

tickers = {
    "bitcoin": "BTC-USD",
    "gold": "GC=F",
    "oil": "CL=F"
}

def download_data(ticker, start, end, interval):
    return yf.download(ticker, start=start, end=end, interval=interval, auto_adjust=True)

btc_daily = download_data(tickers["bitcoin"], start_date_daily, end_date, "1d")
gold_daily = download_data(tickers["gold"], start_date_daily, end_date, "1d")
oil_daily = download_data(tickers["oil"], start_date_daily, end_date, "1d")

btc_1h = download_data(tickers["bitcoin"], start_date_hourly, end_date, "1h")
gold_1h = download_data(tickers["gold"], start_date_hourly, end_date, "1h")
oil_1h = download_data(tickers["oil"], start_date_hourly, end_date, "1h")

btc_4h = download_data(tickers["bitcoin"], start_date_hourly, end_date, "4h")
gold_4h = download_data(tickers["gold"], start_date_hourly, end_date, "4h")
oil_4h = download_data(tickers["oil"], start_date_hourly, end_date, "4h")

btc_daily.to_csv("btc_daily.csv")
gold_daily.to_csv("gold_daily.csv")
oil_daily.to_csv("oil_daily.csv")

btc_1h.to_csv("btc_1h.csv")
gold_1h.to_csv("gold_1h.csv")
oil_1h.to_csv("oil_1h.csv")

btc_4h.to_csv("btc_4h.csv")
gold_4h.to_csv("gold_4h.csv")
oil_4h.to_csv("oil_4h.csv")

def prepare_combined_dataset(btc_df, gold_df, oil_df):
    df = btc_df[['Close']].rename(columns={'Close': 'Close_btc'})
    df['Close_gold'] = gold_df['Close']
    df['Close_oil'] = oil_df['Close']
    df.dropna(inplace=True)
    df['Target'] = df['Close_btc'].shift(-1)
    df.dropna(inplace=True)
    df["Day"] = np.arange(1, len(df)+1)
    return df

df_1h = prepare_combined_dataset(btc_1h, gold_1h, oil_1h)
df_4h = prepare_combined_dataset(btc_4h, gold_4h, oil_4h)
df_1d = prepare_combined_dataset(btc_daily, gold_daily, oil_daily)

# 💾 ذخیره‌سازی فایل‌های نهایی
df_1h.to_csv("df_1h.csv")
df_4h.to_csv("df_4h.csv")
df_1d.to_csv("df_1d.csv")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
import pandas as pd

def load_and_clean_csv(filepath):
    df = pd.read_csv(filepath, skiprows=2)
    expected_cols = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    df.columns = expected_cols[:df.shape[1]]
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)
    return df

btc_1h = load_and_clean_csv("btc_1h.csv")
gold_1h = load_and_clean_csv("gold_1h.csv")
oil_1h = load_and_clean_csv("oil_1h.csv")

btc_4h = load_and_clean_csv("btc_4h.csv")
gold_4h = load_and_clean_csv("gold_4h.csv")
oil_4h = load_and_clean_csv("oil_4h.csv")

btc_1d = load_and_clean_csv("btc_daily.csv")
gold_1d = load_and_clean_csv("gold_daily.csv")
oil_1d = load_and_clean_csv("oil_daily.csv")

btc_1h.info(), btc_1h.head(2)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2161 entries, 2025-03-01 05:00:00+00:00 to 2025-05-30 05:00:00+00:00
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2161 non-null   float64
 1   High       2161 non-null   float64
 2   Low        2161 non-null   float64
 3   Close      2161 non-null   float64
 4   Adj Close  2161 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 101.3 KB


(None,
                                    Open          High           Low  \
 Datetime                                                              
 2025-03-01 05:00:00+00:00  85599.203125  86199.703125  85599.203125   
 2025-03-01 06:00:00+00:00  85017.718750  85614.648438  84933.656250   
 
                                  Close  Adj Close  
 Datetime                                           
 2025-03-01 05:00:00+00:00  86176.78125          0  
 2025-03-01 06:00:00+00:00  85592.25000          0  )

In [5]:
def prepare_full_dataset(btc_df, gold_df, oil_df):
    btc = btc_df.add_prefix('btc_')
    gold = gold_df.add_prefix('gold_')
    oil = oil_df.add_prefix('oil_')

    df = btc.join([gold, oil], how='inner')
    df.dropna(inplace=True)
    df['Day'] = range(1, len(df) + 1)
    df['Target'] = df['btc_Close'].shift(-1)
    df.dropna(inplace=True)
    return df

df_1h = prepare_full_dataset(btc_1h, gold_1h, oil_1h)
df_4h = prepare_full_dataset(btc_4h, gold_4h, oil_4h)
df_1d = prepare_full_dataset(btc_1d, gold_1d, oil_1d)

df_1h.to_csv("df_1h.csv")
df_4h.to_csv("df_4h.csv")
df_1d.to_csv("df_1d.csv")

In [6]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, models, metrics

df_1h = pd.read_csv("df_1h.csv")
df_4h = pd.read_csv("df_4h.csv")
df_1d = pd.read_csv("df_1d.csv")

param_grid_rf = {'n_estimators': [50], 'max_depth': [None]}
param_grid_mlp = {'hidden_layer_sizes': [(50,)], 'max_iter': [500]}
param_grid_dt = {'max_depth': [None]}
param_grid_knn = {'n_neighbors': [3]}
param_grid_ridge = {'alpha': [1.0]}

def perform_grid_search(model, param_grid, X_train, y_train):
    start_time = time.time()
    search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
    search.fit(X_train, y_train)
    train_time = time.time() - start_time
    return search.best_estimator_, train_time

def train_ml(df, timeframe):
    X = df.drop(columns=["Target", "Datetime"])
    y = df["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

    models_dict = {
        "Ridge": (Ridge(), param_grid_ridge),
        "MLP": (MLPRegressor(), param_grid_mlp),
        "LinearRegression": (LinearRegression(), {}),
        "DecisionTree": (DecisionTreeRegressor(), param_grid_dt),
        "KNN": (KNeighborsRegressor(), param_grid_knn),
        "RandomForest": (RandomForestRegressor(), param_grid_rf),
    }

    results = []
    for name, (model, params) in models_dict.items():
        if params:
            best_model, train_time = perform_grid_search(model, params, X_train, y_train)
        else:
            start = time.time()
            model.fit(X_train, y_train)
            best_model = model
            train_time = time.time() - start

        start_pred = time.time()
        preds = best_model.predict(X_test)
        pred_time = time.time() - start_pred
        mae = mean_absolute_error(y_test, preds)
        r2 = r2_score(y_test, preds)
        acc = 1 - (mae / y_test.mean())

        results.append({
            "Model": name,
            "Timeframe": timeframe,
            "TrainTime": train_time,
            "PredictTime": pred_time,
            "MAE": mae,
            "R2": r2,
            "Accuracy": acc
        })
    return results

def sequence_maker(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size, target_index])
    return np.array(X), np.array(y)

def train_rnn(X, y, tf, features):
    X_train, X_test, y_train, y_test = train_test_split(X, y.reshape(-1, 1), test_size=0.1, shuffle=False)

    model = models.Sequential([
        layers.Input(shape=(20, features)),
        layers.SimpleRNN(20, return_sequences=True),
        layers.SimpleRNN(20),
        layers.Dense(1)
    ])
    model.compile(
        loss='mean_squared_error',
        optimizer='adam',
        metrics=[
            metrics.MeanSquaredError(name='mse'),
            metrics.MeanAbsoluteError(name='mae'),
            metrics.MeanAbsolutePercentageError(name='mape'),
            metrics.RootMeanSquaredError(name='rmse'),
            metrics.R2Score(name='r2')
        ]
    )

    start_train = time.time()
    model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), verbose=0)
    train_time = time.time() - start_train

    start_pred = time.time()
    results = model.evaluate(X_test, y_test, verbose=0)
    pred_time = time.time() - start_pred

    return {
        "Model": "RNN",
        "Timeframe": tf,
        "TrainTime": train_time,
        "PredictTime": pred_time,
        "MAE": results[2],
        "R2": results[5],
        "Accuracy": 1 - (results[2] / np.mean(y_test))
    }

def run_rnn(df, tf):
    df = df.drop(columns=["Datetime"])
    target_col = [col for col in df.columns if col.endswith("btc_Close") or col == "Close_btc"]
    target_col = target_col[0] if target_col else "Close_btc"
    target_index = df.columns.get_loc(target_col)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    X, y = sequence_maker(scaled, 20, target_index)
    return train_rnn(X, y, tf, features=X.shape[2])

results = []
for df, tf in zip([df_1h, df_4h, df_1d], ["1h", "4h", "1d"]):
    results.extend(train_ml(df, tf))
    results.append(run_rnn(df, tf))

df_results = pd.DataFrame(results)
print(df_results)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.75416e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=1.3176e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=2.83966e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


               Model Timeframe  TrainTime  PredictTime           MAE  \
0              Ridge        1h   2.736880     0.003717  5.659375e+01   
1                MLP        1h   0.515005     0.003915  5.222718e+04   
2   LinearRegression        1h   0.011583     0.002928  5.666050e+01   
3       DecisionTree        1h   0.184411     0.001998  9.527991e+02   
4                KNN        1h   0.102630     0.033674  1.167362e+04   
5       RandomForest        1h   3.118238     0.023622  9.215000e+02   
6                RNN        1h  18.947366     0.122870  8.999836e-02   
7              Ridge        4h   0.039914     0.001396  6.143221e+01   
8                MLP        4h   0.148139     0.001775  1.112759e+07   
9   LinearRegression        4h   0.002127     0.001254  6.145284e+01   
10      DecisionTree        4h   0.040551     0.001529  1.204403e+03   
11               KNN        4h   0.037931     0.001894  1.855431e+04   
12      RandomForest        4h   0.731764     0.006239  8.896109

In [7]:
import pandas as pd

df_1h = pd.read_csv("df_1h.csv")
df_4h = pd.read_csv("df_4h.csv")
df_1d = pd.read_csv("df_1d.csv")

def process_for_classification(df):
    df = df.drop(columns=["Target"])
    df["Benefit"] = df["btc_Close"] - df["btc_Open"]
    df["Target"] = (df["Benefit"] > 0).astype(int)
    df["Target"] = df["Target"].shift(-1)
    df = df.dropna().reset_index(drop=True)
    return df

df_1h_cls = process_for_classification(df_1h)
df_4h_cls = process_for_classification(df_4h)
df_1d_cls = process_for_classification(df_1d)

df_1h_cls.to_csv("df_1h_class.csv", index=False)
df_4h_cls.to_csv("df_4h_class.csv", index=False)
df_1d_cls.to_csv("df_1d_class.csv", index=False)

In [8]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, models, metrics

df_1h = pd.read_csv("df_1h_class.csv")
df_4h = pd.read_csv("df_4h_class.csv")
df_1d = pd.read_csv("df_1d_class.csv")

param_grid_rf = {'n_estimators': [50], 'max_depth': [None]}
param_grid_mlp = {'hidden_layer_sizes': [(50,)], 'max_iter': [500]}
param_grid_dt = {'max_depth': [None]}
param_grid_knn = {'n_neighbors': [3]}

def perform_grid_search(model, param_grid, X_train, y_train):
    start_time = time.time()
    search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    search.fit(X_train, y_train)
    train_time = time.time() - start_time
    return search.best_estimator_, train_time

def train_classification_models(df, timeframe):
    X = df.drop(columns=["Target", "Datetime"])
    y = df["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

    models_dict = {
        "LogisticRegression": (LogisticRegression(max_iter=500), {}),
        "MLP": (MLPClassifier(), param_grid_mlp),
        "DecisionTree": (DecisionTreeClassifier(), param_grid_dt),
        "KNN": (KNeighborsClassifier(), param_grid_knn),
        "RandomForest": (RandomForestClassifier(), param_grid_rf),
    }

    results = []
    for name, (model, params) in models_dict.items():
        if params:
            best_model, train_time = perform_grid_search(model, params, X_train, y_train)
        else:
            start = time.time()
            model.fit(X_train, y_train)
            best_model = model
            train_time = time.time() - start

        start_pred = time.time()
        preds = best_model.predict(X_test)
        pred_time = time.time() - start_pred

        results.append({
            "Model": name,
            "Timeframe": timeframe,
            "TrainTime": train_time,
            "PredictTime": pred_time,
            "Accuracy": accuracy_score(y_test, preds),
            "Precision": precision_score(y_test, preds, zero_division=0),
            "Recall": recall_score(y_test, preds, zero_division=0),
            "F1": f1_score(y_test, preds, zero_division=0)
        })
    return results

def sequence_maker(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size, target_index])
    return np.array(X), np.array(y)

def train_rnn_classifier(X, y, tf, features):
    X_train, X_test, y_train, y_test = train_test_split(X, y.reshape(-1, 1), test_size=0.1, shuffle=False)

    model = models.Sequential([
        layers.Input(shape=(20, features)),
        layers.SimpleRNN(20, return_sequences=True),
        layers.SimpleRNN(20),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=[
            metrics.BinaryAccuracy(name='accuracy'),
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')
        ]
    )

    start_train = time.time()
    model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), verbose=0)
    train_time = time.time() - start_train

    start_pred = time.time()
    eval_result = model.evaluate(X_test, y_test, verbose=0)
    pred_time = time.time() - start_pred

    return {
        "Model": "RNN",
        "Timeframe": tf,
        "TrainTime": train_time,
        "PredictTime": pred_time,
        "Accuracy": eval_result[1],
        "Precision": eval_result[2],
        "Recall": eval_result[3],
        "F1": 2 * eval_result[2] * eval_result[3] / (eval_result[2] + eval_result[3] + 1e-10)
    }

def run_rnn(df, tf):
    df = df.drop(columns=["Datetime"])
    target_col = "Target"
    target_index = df.columns.get_loc(target_col)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    X, y = sequence_maker(scaled, 20, target_index)
    return train_rnn_classifier(X, y, tf, features=X.shape[2])

results = []
for df, tf in zip([df_1h, df_4h, df_1d], ["1h", "4h", "1d"]):
    results.extend(train_classification_models(df, tf))
    results.append(run_rnn(df, tf))

df_results_cls = pd.DataFrame(results)
print(df_results_cls)

                 Model Timeframe  TrainTime  PredictTime  Accuracy  Precision  \
0   LogisticRegression        1h   0.032365     0.002019  0.478873   0.000000   
1                  MLP        1h   0.300668     0.001796  0.514085   0.524752   
2         DecisionTree        1h   0.313549     0.018931  0.485915   0.507937   
3                  KNN        1h   0.296577     0.023079  0.507042   0.531250   
4         RandomForest        1h   1.163121     0.007475  0.443662   0.463768   
5                  RNN        1h  13.490292     0.120205  0.478571   0.512821   
6   LogisticRegression        4h   0.019569     0.003134  0.416667   0.437500   
7                  MLP        4h   0.154250     0.002010  0.500000   0.515152   
8         DecisionTree        4h   0.053378     0.001791  0.611111   0.647059   
9                  KNN        4h   0.051696     0.005382  0.444444   0.454545   
10        RandomForest        4h   0.495700     0.008265  0.472222   0.500000   
11                 RNN      

In [9]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, models, metrics

df_1h = pd.read_csv("df_1h_class.csv")
df_4h = pd.read_csv("df_4h_class.csv")
df_1d = pd.read_csv("df_1d_class.csv")


param_grid_rf = {'n_estimators': [50], 'max_depth': [None]}
param_grid_mlp = {'hidden_layer_sizes': [(50,)], 'max_iter': [500]}
param_grid_dt = {'max_depth': [None]}
param_grid_knn = {'n_neighbors': [3]}

def perform_grid_search(model, param_grid, X_train, y_train):
    start_time = time.time()
    search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    search.fit(X_train, y_train)
    train_time = time.time() - start_time
    return search.best_estimator_, train_time

def train_classification_models(df, timeframe):
    X = df.drop(columns=["Target", "Datetime"])
    y = df["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

    models_dict = {
        "LogisticRegression": (LogisticRegression(max_iter=500), {}),
        "MLP": (MLPClassifier(), param_grid_mlp),
        "DecisionTree": (DecisionTreeClassifier(), param_grid_dt),
        "KNN": (KNeighborsClassifier(), param_grid_knn),
        "RandomForest": (RandomForestClassifier(), param_grid_rf),
    }

    results = []
    for name, (model, params) in models_dict.items():
        if params:
            best_model, train_time = perform_grid_search(model, params, X_train, y_train)
        else:
            start = time.time()
            model.fit(X_train, y_train)
            best_model = model
            train_time = time.time() - start

        start_pred = time.time()
        preds = best_model.predict(X_test)
        pred_time = time.time() - start_pred

        results.append({
            "Model": name,
            "Timeframe": timeframe,
            "TrainTime": train_time,
            "PredictTime": pred_time,
            "Accuracy": accuracy_score(y_test, preds),
            "Precision": precision_score(y_test, preds, zero_division=0),
            "Recall": recall_score(y_test, preds, zero_division=0),
            "F1": f1_score(y_test, preds, zero_division=0)
        })
    return results

def sequence_maker(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size, target_index])
    return np.array(X), np.array(y)

def train_rnn_classifier(X, y, tf, features):
    from sklearn.metrics import precision_score, recall_score, f1_score

    X_train, X_test, y_train, y_test = train_test_split(X, y.reshape(-1, 1), test_size=0.1, shuffle=False)

    model = models.Sequential([
        layers.Input(shape=(20, features)),
        layers.SimpleRNN(20, return_sequences=True),
        layers.SimpleRNN(20),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=[
            metrics.BinaryAccuracy(name='accuracy'),
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')
        ]
    )

    start_train = time.time()
    model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test), verbose=0)
    train_time = time.time() - start_train

    start_pred = time.time()
    y_pred_prob = model.predict(X_test)
    y_pred_class = (y_pred_prob > 0.5).astype(int)
    pred_time = time.time() - start_pred

    preds_df = pd.DataFrame({
        'True': y_test.flatten(),
        'Predicted': y_pred_class.flatten(),
        'Probability': y_pred_prob.flatten()
    })
    preds_df.to_csv(f"rnn_preds_{tf}.csv", index=False)

    acc = np.mean(y_pred_class.flatten() == y_test.flatten())
    prec = precision_score(y_test, y_pred_class, zero_division=0)
    rec = recall_score(y_test, y_pred_class, zero_division=0)
    f1 = f1_score(y_test, y_pred_class, zero_division=0)

    return {
        "Model": "RNN",
        "Timeframe": tf,
        "TrainTime": train_time,
        "PredictTime": pred_time,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1
    }

def run_rnn(df, tf):
    df = df.drop(columns=["Datetime"])
    target_col = "Target"
    target_index = df.columns.get_loc(target_col)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)
    X, y = sequence_maker(scaled, 20, target_index)
    return train_rnn_classifier(X, y, tf, features=X.shape[2])

results = []
for df, tf in zip([df_1h, df_4h, df_1d], ["1h", "4h", "1d"]):
    results.extend(train_classification_models(df, tf))
    results.append(run_rnn(df, tf))

df_results_cls = pd.DataFrame(results)
print(df_results_cls)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 349ms/step


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step
                 Model Timeframe  TrainTime  PredictTime  Accuracy  Precision  \
0   LogisticRegression        1h   0.018364     0.002264  0.478873   0.000000   
1                  MLP        1h   0.327026     0.001939  0.542254   0.666667   
2         DecisionTree        1h   0.145365     0.002056  0.542254   0.558442   
3                  KNN        1h   0.094268     0.011659  0.507042   0.531250   
4         RandomForest        1h   1.105934     0.006121  0.507042   0.520833   
5                  RNN        1h  19.820264     0.648466  0.478571   0.516129   
6   LogisticRegression        4h   0.024432     0.002319  0.416667   0.437500   
7                  MLP        4h   0.173702     0.002001  0.527778   1.000000   
8         DecisionTree        4h   0.053479     0.002269  0.583333   0.600000   
9                  KNN        4h   0.063440     0.006646  0.444444   0.454545   
10        RandomForest        4h   0.488087     0.005486  0.361111   0.